# MLP Coding Example

Acesse o servidor remoto por ssh. Crie uma virtualenv com:
```
mkvirtualenv <nome-da-sua-env>
```
Ative a sua virtualenv com:
```
workon <nome-da-sua-env>
```
Instale o jupyter:
```
pip install jupyter
```
Na pasta contendo o setup.py, instale o pacote do projeto :
```
pip install -e .
```
Comando para servir o jupyter:
```
nohup jupyter notebook --no-browser &
```
Talvez você precise de um token. Se precisar consulte com:
```
jupyter notebook list
```




Na sua máquina local, redirecione a porta adequada:
```
ssh -NfL localhost:<porta-local>:localhost:<porta-remoto> <seu-usuario>@<ip-do-servidor>
```
Geralmente:
```
ssh -NfL localhost:8888:localhost:8888 <seu-usuario>@<ip-do-servidor>
```
Abra localhost:8888 no seu browser. Se você quiser fechar o jupyter, no localhost:8888 clique em Quit, depois libere a porta com:
```
lsof -ti:8888 | xargs kill -9
```

## Imports

In [1]:
import numpy as np
from numpy import genfromtxt
from tqdm.notebook import tqdm
from perceptronac.context_training import context_training
from perceptronac.context_coding import context_coding
from perceptronac.perfect_AC import perfect_AC
import torch
import os

## Gerando dados randômicos correlacionados (substituir pelos seus dados)

In [2]:
#X is context and y is bitstream to encode
# parameters  
L = 100000 # how many samples 
N = 7 # order of the AR
# Np = N # number of parameters to estimate 

C0 = np.random.rand(1,1) 
C = np.random.rand(N,1)

X = 2 * (np.random.rand(2*L,N) > 0.5) - 1 # correlated (context) signals

X = (X > 0).astype(int)

def sigmoid(x): 
    return 1 / (1 + np.e**(-x))

p = sigmoid(C0 + X @ C);
yy = (np.random.rand(2*L, 1) > (1 - p)).astype(int) # signal 
yt = yy[0:L] > 0 # train on the first part 
yc = yy[L:L+L] > 0 # encode the second part
Xt = X[0:L,0:N] # truncated X for training 
Xc = X[L:L+L,0:N] # truncated X for coding

In [3]:
#Xc

In [2]:
#X is context and y is bitstream to encode
L = 100000 # how many samples 
#N = 10 # order of the AR
first = True
for pc_num in range(11, 38):
    if first:
        my_data = genfromtxt(f'../SPIHT_dataset/SPIHT_bits_with_context_ricardo{pc_num}_v5.csv', delimiter=',')
        first = False
    else:
        my_data = np.append(my_data,
                            genfromtxt(f'../SPIHT_dataset/SPIHT_bits_with_context_ricardo{pc_num}_v5.csv', delimiter=','),
                            axis=0)
    
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:6].astype(float)
extra_context = my_data[:, 6:].astype(float)
#extra_context = extra_context.reshape((len(extra_context), 1))
bitstream = bitstream.reshape((len(bitstream), 1))


my_data_test = genfromtxt('../SPIHT_dataset/SPIHT_bits_with_context_ricardo39_v5.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 6:].astype(float)
#extra_context_test = extra_context_test.reshape((len(extra_context_test), 1))
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))


yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context >= 0 # truncated X for training 
Xc = context_test >= 0 # truncated X for coding
Xt = Xt.astype(int)
Xc = Xc.astype(int)
Xt = np.append(Xt, extra_context/extra_context.max(axis=0), axis=1)
Xc = np.append(Xc, extra_context_test/extra_context_test.max(axis=0), axis=1)

N = Xc[0, :].size # order of the AR

In [3]:
Xc[0:6, :]

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.03703704, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.03703704, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.03703704, 1.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        0.03703704, 1.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.03703704, 1.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0. 

## Entropia dos dados

In [4]:
# treino
perfect_AC(yt,context_coding(Xt,context_training(Xt,yt)))

0.863930727603271

In [5]:
# teste
perfect_AC(yc[:],context_coding(Xc[:, :],context_training(Xc[:, :],yc[:])))

0.8630416694804717

# Criando classes

In [6]:
import torch

In [7]:
class Perceptron(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.linear = torch.nn.Linear(N, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [8]:
class Log2BCELoss(torch.nn.Module):
    def __init__(self,*args,**kwargs):
        super().__init__()
        self.bce_loss = torch.nn.BCELoss(*args,**kwargs)

    def forward(self, pred, target):
        return self.bce_loss(pred, target)/torch.log(torch.tensor(2,dtype=target.dtype,device=target.device))

In [9]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx,:]

In [10]:
class MLP_N_64N_32N_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 64*N),
            torch.nn.ReLU(),
            torch.nn.Linear(64*N, 32*N),
            torch.nn.ReLU(),
            torch.nn.Linear(32*N, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

In [11]:
class MLP_N_1024_512_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)


class MLP_N_2048_1024_1(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(N, 2048),
            torch.nn.ReLU(),
            torch.nn.Linear(2048, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        return self.layers(x)

## Treinando Modelo No Pytorch com Batch Gradient Descent (Quando todos os dados couberem na memória da placa de vídeo de uma só vez)

In [61]:
net = Perceptron(N)

In [62]:
net = net.cuda()

In [63]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [64]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [65]:
train_loss = []
for epoch in range(20000):
    optimizer.zero_grad()
    outputs = net(torch.tensor(trainset.X).float().cuda())
    loss = criterion(outputs,torch.tensor(trainset.y).view(-1,1).float().cuda())
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item()/len(trainset))

In [66]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

comprimento médio de código final no dataset de treino: 0.003491003723144531
(compare com a entropia do dataset de treino).


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [67]:
for param in net.parameters():
    print(param.data)

tensor([[12.0901, 11.7186, 11.9979, 12.1763, 12.1026]], device='cuda:0')
tensor([41.8704], device='cuda:0')


In [68]:
C.T, C0

(array([[0.32098451, 0.77295341, 0.82311742, 0.86189127, 0.7005142 ,
         0.80153656, 0.21184858]]),
 array([[0.2792105]]))

## Treinando Modelo No Pytorch com Stochastic Gradient Descent (um pedaço dos dados na memória da placa de vídeo de cada vez)

In [12]:
#net = Perceptron(N)
#net = MLP_N_64N_32N_1(N)
net = MLP_N_1024_512_1(N) #melhor resultado
#net = MLP_N_2048_1024_1(N)

In [13]:
net = net.cuda()

In [14]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [15]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.000001)

In [16]:

batch_size = 100000

train_loss, valid_loss = [], []

for epoch in range(400):  # loop over the dataset multiple times

    for phase in ['train','valid']:

        if phase == 'train':
            net.train(True)
            dataloader = torch.utils.data.DataLoader(
                trainset,batch_size=batch_size,shuffle=True,num_workers=2)
        else:
            net.train(False)
            dataloader=torch.utils.data.DataLoader(
                validset,batch_size=batch_size,shuffle=False,num_workers=2)
            
        running_loss = 0.0
        for data in dataloader: #tqdm(dataloader):
            
            X_b,y_b= data
            X_b = X_b.float().cuda()
            y_b = y_b.float().cuda()
            
            if phase == 'train':
                optimizer.zero_grad()
                outputs = net(X_b.float())
                loss = criterion(outputs,y_b.view(-1,1).float())
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs = net(X_b)
                    loss = criterion(outputs, y_b)

            running_loss += loss.item()

        final_loss = running_loss / len(dataloader.dataset)
        if phase=='train':
            train_loss.append(final_loss)
        else:
            valid_loss.append(final_loss)
            
        print("epoch :" , epoch, ", phase :", phase, ", loss :", final_loss)

print('Finished Training')

epoch : 0 , phase : train , loss : 0.94574865010092
epoch : 0 , phase : valid , loss : 0.9375248378642078
epoch : 1 , phase : train , loss : 0.9390850375099759
epoch : 1 , phase : valid , loss : 0.9302343690444428
epoch : 2 , phase : train , loss : 0.9335460590307181
epoch : 2 , phase : valid , loss : 0.9266530133035238
epoch : 3 , phase : train , loss : 0.9292743493473878
epoch : 3 , phase : valid , loss : 0.9222399453815845
epoch : 4 , phase : train , loss : 0.9260195367711569
epoch : 4 , phase : valid , loss : 0.9189252337641299
epoch : 5 , phase : train , loss : 0.9232785168124248
epoch : 5 , phase : valid , loss : 0.9158085710125332
epoch : 6 , phase : train , loss : 0.9205771347927059
epoch : 6 , phase : valid , loss : 0.9134357227979072
epoch : 7 , phase : train , loss : 0.9187165659181135
epoch : 7 , phase : valid , loss : 0.9121008005927992
epoch : 8 , phase : train , loss : 0.9171505613955967
epoch : 8 , phase : valid , loss : 0.9103757255783016
epoch : 9 , phase : train , lo

epoch : 75 , phase : train , loss : 0.8895677473723284
epoch : 75 , phase : valid , loss : 0.8894611724467335
epoch : 76 , phase : train , loss : 0.9034983581073902
epoch : 76 , phase : valid , loss : 0.8912086844914227
epoch : 77 , phase : train , loss : 0.8896750996841278
epoch : 77 , phase : valid , loss : 0.8895827083543196
epoch : 78 , phase : train , loss : 0.9041732531572115
epoch : 78 , phase : valid , loss : 0.908502341235904
epoch : 79 , phase : train , loss : 0.9067968403181704
epoch : 79 , phase : valid , loss : 0.9040226614696205
epoch : 80 , phase : train , loss : 0.905291027265938
epoch : 80 , phase : valid , loss : 0.9023425987695138
epoch : 81 , phase : train , loss : 0.9040644940785691
epoch : 81 , phase : valid , loss : 0.9004829361373685
epoch : 82 , phase : train , loss : 0.9029609854528674
epoch : 82 , phase : valid , loss : 0.899228408637668
epoch : 83 , phase : train , loss : 0.9011974786937679
epoch : 83 , phase : valid , loss : 0.8961663402666796
epoch : 84 , 

epoch : 149 , phase : train , loss : 0.8888545597529842
epoch : 149 , phase : valid , loss : 0.889461188399119
epoch : 150 , phase : train , loss : 0.8889325871542175
epoch : 150 , phase : valid , loss : 0.8892730725519596
epoch : 151 , phase : train , loss : 0.8885610573832429
epoch : 151 , phase : valid , loss : 0.8891287912013959
epoch : 152 , phase : train , loss : 0.8885151774439978
epoch : 152 , phase : valid , loss : 0.8888804896625139
epoch : 153 , phase : train , loss : 0.8882935500770193
epoch : 153 , phase : valid , loss : 0.888737367518629
epoch : 154 , phase : train , loss : 0.888270593798725
epoch : 154 , phase : valid , loss : 0.888714409377178
epoch : 155 , phase : train , loss : 0.8888204681054506
epoch : 155 , phase : valid , loss : 0.8886405285625123
epoch : 156 , phase : train , loss : 0.8880953665216886
epoch : 156 , phase : valid , loss : 0.8884558477956951
epoch : 157 , phase : train , loss : 0.8999454709773118
epoch : 157 , phase : valid , loss : 0.9041944660026

epoch : 222 , phase : valid , loss : 0.8869225470420959
epoch : 223 , phase : train , loss : 0.8859560363979961
epoch : 223 , phase : valid , loss : 0.887042173980885
epoch : 224 , phase : train , loss : 0.8860452596064876
epoch : 224 , phase : valid , loss : 0.8871861442599318
epoch : 225 , phase : train , loss : 0.8859270800634127
epoch : 225 , phase : valid , loss : 0.8868644192081014
epoch : 226 , phase : train , loss : 0.8859974059342212
epoch : 226 , phase : valid , loss : 0.8871068609040468
epoch : 227 , phase : train , loss : 0.885943177873681
epoch : 227 , phase : valid , loss : 0.8866277496169725
epoch : 228 , phase : train , loss : 0.8859573338008045
epoch : 228 , phase : valid , loss : 0.8868142994716229
epoch : 229 , phase : train , loss : 0.8860206321690005
epoch : 229 , phase : valid , loss : 0.886827779237362
epoch : 230 , phase : train , loss : 0.8858346016440295
epoch : 230 , phase : valid , loss : 0.8867092822592118
epoch : 231 , phase : train , loss : 0.885949940600

epoch : 296 , phase : train , loss : 0.885339121362563
epoch : 296 , phase : valid , loss : 0.8864591222674202
epoch : 297 , phase : train , loss : 0.8853711644490516
epoch : 297 , phase : valid , loss : 0.8862964664275028
epoch : 298 , phase : train , loss : 0.8852887503988373
epoch : 298 , phase : valid , loss : 0.8863598346201307
epoch : 299 , phase : train , loss : 0.885404023928464
epoch : 299 , phase : valid , loss : 0.8861702325423899
epoch : 300 , phase : train , loss : 0.8853325564506294
epoch : 300 , phase : valid , loss : 0.8863112463126592
epoch : 301 , phase : train , loss : 0.8853538749545414
epoch : 301 , phase : valid , loss : 0.8863769701408779
epoch : 302 , phase : train , loss : 0.8852428932917327
epoch : 302 , phase : valid , loss : 0.8861798731006877
epoch : 303 , phase : train , loss : 0.8852890995962804
epoch : 303 , phase : valid , loss : 0.886376058196174
epoch : 304 , phase : train , loss : 0.885217303067009
epoch : 304 , phase : valid , loss : 0.8862778207475

epoch : 369 , phase : valid , loss : 0.8858942642496066
epoch : 370 , phase : train , loss : 0.8848389265318787
epoch : 370 , phase : valid , loss : 0.8857951786658667
epoch : 371 , phase : train , loss : 0.8849900263761186
epoch : 371 , phase : valid , loss : 0.8862041871949053
epoch : 372 , phase : train , loss : 0.8848596999424191
epoch : 372 , phase : valid , loss : 0.8862097386250558
epoch : 373 , phase : train , loss : 0.8848765592717271
epoch : 373 , phase : valid , loss : 0.885837484392186
epoch : 374 , phase : train , loss : 0.8848455027639491
epoch : 374 , phase : valid , loss : 0.8861591642455908
epoch : 375 , phase : train , loss : 0.8849234892975696
epoch : 375 , phase : valid , loss : 0.885703125425397
epoch : 376 , phase : train , loss : 0.8849210779165437
epoch : 376 , phase : valid , loss : 0.8859937513017146
epoch : 377 , phase : train , loss : 0.8848914667408208
epoch : 377 , phase : valid , loss : 0.8861354802706001
epoch : 378 , phase : train , loss : 0.88481365211

In [17]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

print(f"""comprimento médio de código final no dataset de validação: {valid_loss[-1]}
(compare com a entropia do dataset de validação).""")

comprimento médio de código final no dataset de treino: 0.8846855897134679
(compare com a entropia do dataset de treino).
comprimento médio de código final no dataset de validação: 0.8859299975931041
(compare com a entropia do dataset de validação).


### Salva os pesos

In [18]:
file_name = 'pesos_ricardo9-v3_1024-512.txt'

In [19]:
torch.save(net.eval().state_dict(), file_name)

In [21]:
model = MLP_N_1024_512_1(N)
model.load_state_dict(torch.load(file_name))

<All keys matched successfully>

In [22]:
my_data_test = genfromtxt('../SPIHT_dataset/SPIHT_bits_with_context_ricardo40_v5.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 6:].astype(float)
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))

yc = bitstream_test > 0 # encode the second part
Xc = context_test >= 0 # truncated X for coding
Xc = Xc.astype(int)
Xc = np.append(Xc, extra_context_test/extra_context_test.max(axis=0), axis=1)

In [29]:
Xc_tensor = torch.from_numpy(Xc[0:9357, :])
yc_tensor = torch.from_numpy(yc[0:9357])
predictions = model(Xc_tensor.float())
criterion=Log2BCELoss()
tamanho_medio = criterion(predictions,yc_tensor.float())
print(f"""comprimento médio de código final no modelo treinado: {tamanho_medio}""")

comprimento médio de código final no modelo treinado: 0.9560299515724182


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [ ]:
for param in net.parameters():
    print(param.data)

In [17]:
C.T, C0

NameError: name 'C' is not defined